In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Imports

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Reading csv files

In [ ]:
#Users
u_cols = ['user_id', 'location', 'age']
users = pd.read_csv('../input/bookcrossing-dataset/Book reviews/BX-Users.csv', sep=';', names=u_cols, encoding='latin-1',low_memory=False)

#Books
i_cols = ['isbn', 'book_title' ,'book_author','year_of_publication', 'publisher', 'img_s', 'img_m', 'img_l']
items = pd.read_csv('../input/bookcrossing-dataset/Book reviews/BX-Books.csv', sep=';', names=i_cols, encoding='latin-1',low_memory=False)

#Ratings
r_cols = ['user_id', 'isbn', 'rating']
ratings = pd.read_csv('../input/bookcrossing-dataset/Book reviews/BX-Book-Ratings.csv', sep=';', names=r_cols, encoding='latin-1',low_memory=False)

# What does the data look like?

In [ ]:
users.head(5)
users.shape

In [ ]:
items.head(5)
items.shape

In [ ]:
ratings.head(5)
ratings.shape

In [ ]:
users = users.drop(users.index[0])
items = items.drop(items.index[0])
ratings = ratings.drop(ratings.index[0])

# Merging the dataframes

In [ ]:
df = pd.merge(users, ratings, on='user_id')
df = pd.merge(df, items, on='isbn')
df.head(5)
df.shape

Changing The Dataframe chunk to 1 Lakh because I was running out of memory. you can work with original data which will give you better result.

In [ ]:
df = df[:102000]
df.shape

# Dropping null values and Unwanted values

In [ ]:
df = df.dropna()
df.drop(['img_s','img_m','img_l'],axis=1,inplace=True)
df.head()

In [ ]:
df.describe()

# Changing datatypes

In [ ]:
df['age'] = df['age'].astype(int)
df['user_id'] = df['user_id'].astype(int)
df['rating'] = df['rating'].astype(int)

## Splitting location column into newer columns

In [ ]:
location = df.location.str.split(', ', n=2, expand=True)
location.columns=['city', 'state', 'country']

df['city'] = location['city']
df['state'] = location['state']
df['country'] = location['country']

In [ ]:
df.head(5)

In [ ]:
df.isna().sum()

Lets see which books got highest ratings. As we can see here these 5 books from top got 10/10 rating. Bit suspicious, it is possible that only one user read it and rate it as 10.

In [ ]:
df.groupby('book_title')['rating'].mean().sort_values(ascending=False).head(5)

So now we use the count() function to see the number of user that rated the books. In this we can see 'Wild Animus' has the most users rated to it.

In [ ]:
df.groupby('book_title')['rating'].count().sort_values(ascending=False).head(5)

In [ ]:
ratings = pd.DataFrame(df.groupby('book_title')['rating'].mean())
ratings.head(5)

In [ ]:
ratings['num of ratings'] = pd.DataFrame(df.groupby('book_title')['rating'].count())
ratings.head(5)

In [ ]:
ratings['num of ratings'].hist(bins=80)

In [ ]:
ratings['rating'].hist(bins=70)

In [ ]:
sns.jointplot(x='rating',y='num of ratings',data=ratings,alpha=0.5)

In [ ]:
df.head(10)

### We now create a pivot table. From this table we use the same users who rated for different books. This is collaborative filtering based recommendation system(similarity between users). 

In [ ]:
df1 = df.pivot_table(index='user_id',columns='book_title',values='rating').fillna(0)
df1.head(10)

In [ ]:
ratings.sort_values('num of ratings',ascending=False).head(10)

# Collaborative Filtering Based Recommender

### DV_rating - The Da Vinci Code, LP_rating - Life of Pi, HP_rating - Harry Potter. I'm using these three books for our recomendation. 

In [ ]:
DV_rating = df1['The Da Vinci Code']
LP_rating = df1['Life of Pi']
HP_rating = df1['Harry Potter and the Goblet of Fire (Book 4)']
DV_rating.head()

In [ ]:
similar_to_DV = df1.corrwith(DV_rating)
similar_to_LP = df1.corrwith(LP_rating)
similar_to_HP = df1.corrwith(HP_rating)
similar_to_DV.sort_values(ascending=False).head()

In [ ]:
corr_DV = pd.DataFrame(similar_to_DV,columns=['correlation'])
corr_DV.dropna(inplace=True)
corr_DV = corr_DV.join(ratings['num of ratings'])
corr_DV.head()

### We can see different books suggested whose number of ratings are less but has perfect correlation. So lets filter out the number of ratings to 200. This will improve our recommended books.

In [ ]:
corr_DV[corr_DV['num of ratings']>200].sort_values('correlation',ascending=False).head()

In [ ]:
corr_LP = pd.DataFrame(similar_to_LP,columns=['correlation'])
corr_LP.dropna(inplace=True)
corr_LP = corr_LP.join(ratings['num of ratings'])

In [ ]:
corr_LP[corr_LP['num of ratings']>200].sort_values('correlation',ascending=False).head()

In [ ]:
corr_HP = pd.DataFrame(similar_to_HP,columns=['correlation'])
corr_HP.dropna(inplace=True)
corr_HP = corr_HP.join(ratings['num of ratings'])

In [ ]:
corr_HP[corr_HP['num of ratings']>200].sort_values('correlation',ascending=False).head()